In [1]:
from collections import defaultdict
from tqdm import tqdm_notebook
import json
import string
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
stop_words = set(stopwords.words('english'))
punc = set()


In [2]:
for w in list(string.punctuation):
    punc.add(w)

  
stop_words_more = stop_words.union(punc)

In [3]:
stop_words_more = stop_words_more.union('\n')

In [4]:
stop_words_more

{'\n',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',

In [5]:
user_comments = defaultdict(list)
count = 0
with open('data/comments_final') as infile:
    for line in tqdm_notebook(infile):
        count+=1
        if count > 10000000:
            break
        comment = json.loads(line)
        filtered_sentence= []
        word_tokens = comment['body'].split()
        filtered_sentence = [w.lower() for w in word_tokens if not w in stop_words_more]
#         filtered_sentence = [w for w in filtered_sentence if not w == '\n']
#         filtered_sentence = [w for w in filtered_sentence if not w in string.punctuation]
#         filtered_sentence = [w.lower() for w in filtered_sentence]
#         user = comment['user']
        if len(user_comments[comment['user']]) < 250:
            user_comments[comment['user']].extend(filtered_sentence)

In [6]:
user_comments_new = defaultdict(list)
for user in user_comments:
    if len(user_comments[user]) > 200:
        user_comments_new[user] = user_comments[user]
        

In [7]:
len(user_comments_new)

226501

In [5]:
# count = 0
# with open('comments_final') as infile:
#     for line in tqdm_notebook(infile):
#         count+=1
#         if(count>11500000):
#             break
#         comment = json.loads(line)
#         if ((comment['user'] not in user_comments) or (len(user_comments[comment['user']]) < 200)):  
#             filtered_sentence= []
#             word_tokens = comment['body'].split()
#             filtered_sentence = [w.lower() for w in word_tokens if not w in stop_words_more]
#             user_comments[comment['user']].extend(filtered_sentence)

NameError: name 'user_comments' is not defined

In [8]:
# len(user_comments)

577667

In [8]:
def save_to_file(doc_vecs, filename):
    vecs = []
    for key in user_comments_new:
        vecs.append((key, list(map(str, doc_vecs[key]))))
       
    with open(filename, 'w') as f:
        f.write(json.dumps(vecs))

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# from gensim.models import epoch
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    
    def _init_(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [14]:
def generate_models():
    vector_sizes = [16]
    for v_size in tqdm_notebook(vector_sizes):
        epoch_logger = EpochLogger()
        model = Doc2Vec(tagged_data, vector_size=v_size, window=2, min_count=5, epochs=40, workers=8)
        save_to_file(doc_vecs=model.docvecs, filename='user_' + str(v_size))

In [15]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


In [16]:
tagged_data = [TaggedDocument(words=(_d), tags=[str(i)]) for i,_d in user_comments_new.items()]

In [17]:
generate_models()

In [18]:
model

NameError: name 'model' is not defined

In [19]:
user_features = {}
with open('user_16') as f:
    for line in tqdm_notebook(f):
        item = json.loads(line)
        user_features[item[0]] = np.array(list(map(float, item[1])))

NameError: name 'np' is not defined